## Using Gensim for Topic Modeling

We’re going to first study the gensim implementations because they offer more functionality out of the box and then we’ll replicate that functionality with sklearn. Let’s first prepare the dataset we’ll be working with.


In [1]:
import nltk
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re 

import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords


C:\Users\Urfah\Anaconda3\envs\nlptrain\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
article = open('Berita.txt', encoding="utf8").read().split('BERHENTI DISINI')
len(article)

32

Clean the data from html tags with ``beautifulsoup``

In [3]:
article_clean = []
for text in article:
    text = BeautifulSoup(text, 'html.parser').getText()
    article_clean.append(text)
article = article_clean
print(article[0][:100])



Kroasia: Melawan Argentina adalah Pertandingan Termudah





Jakarta, CNN Indonesia -- Agung Rahma


Tokenize and clean stopwords

In [4]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [5]:
def tokenize_and_stem(text):
    stopwords = nltk.corpus.stopwords.words('indonesian')
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and token not in stopwords:
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [6]:
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in article:
    tokenized_data.append(tokenize_and_stem(text))

# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...


[(2, 1), (3, 1), (11, 1), (19, 4), (22, 2), (24, 1), (26, 1), (44, 1), (50, 1), (164, 2), (183, 9), (196, 1), (223, 2), (230, 1), (252, 4), (274, 1), (280, 1), (284, 1), (309, 1), (314, 1), (335, 2), (341, 1), (404, 1), (431, 1), (434, 1), (452, 1), (465, 1), (474, 1), (480, 1), (485, 2), (489, 1), (500, 1), (504, 1), (520, 1), (523, 1), (534, 1), (538, 2), (546, 1), (547, 9), (552, 1), (596, 1), (597, 3), (599, 1), (619, 2), (676, 1), (845, 1), (904, 1), (914, 2), (927, 1), (932, 1), (967, 1), (1032, 1), (1038, 2), (1165, 2), (1174, 1), (1283, 1), (1397, 1), (1398, 8), (1399, 1), (1400, 1), (1401, 3), (1402, 1), (1403, 1), (1404, 2), (1405, 4), (1406, 1), (1407, 2), (1408, 3), (1409, 1), (1410, 1), (1411, 1), (1412, 3), (1413, 1), (1414, 4), (1415, 2), (1416, 1), (1417, 1), (1418, 1), (1419, 1), (1420, 1), (1421, 1), (1422, 1), (1423, 1), (1424, 2), (1425, 1), (1426, 1), (1427, 1), (1428, 1), (1429, 1), (1430, 1), (1431, 4), (1432, 1), (1433, 1), (1434, 1), (1435, 1), (1436, 1), (1437

In [7]:
NUM_TOPICS = 3

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

We’re going to run LDA and LSI (Latent Semantic Indexing AKA Latent Semantic Analysis) models, which implementation included in the gensim package.

Let’s now display the topics the two models have inferred:

In [8]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.018*"persen" + 0.006*"diskon" + 0.006*"indonesia" + 0.005*"cnn" + 0.005*"lapor" + 0.005*"belanja" + 0.005*"balap" + 0.005*"uang" + 0.005*"triliun" + 0.004*"hasil"
Topic #1: 0.018*"persen" + 0.006*"lemah" + 0.006*"indonesia" + 0.005*"jakarta" + 0.005*"dolar" + 0.005*"as" + 0.005*"naik" + 0.005*"bunga" + 0.005*"minus" + 0.004*"dunia"
Topic #2: 0.010*"persen" + 0.007*"jakarta" + 0.007*"main" + 0.006*"indonesia" + 0.006*"dunia" + 0.005*"duga" + 0.005*"tanding" + 0.005*"oknum" + 0.005*"laku" + 0.004*"novel"
LSI Model:
Topic #0: 0.731*"persen" + 0.216*"lemah" + 0.171*"dolar" + 0.165*"minus" + 0.133*"indeks" + 0.128*"bunga" + 0.116*"kuat" + 0.111*"uang" + 0.104*"as" + 0.103*"dagang"
Topic #1: -0.385*"novel" + -0.311*"oknum" + -0.300*"duga" + -0.271*"jenderal" + -0.165*"kpk" + -0.142*"main" + -0.141*"keras" + 0.139*"persen" + -0.136*"polisi" + -0.131*"air"
Topic #2: 0.391*"main" + 0.294*"tanding" + 0.248*"dunia" + 0.242*"argentina" + 0.230*"lawan" + -0.202*"novel" + 0.19

Let’s now put the models to work and transform unseen documents to their topic distribution:

In [9]:
text = "Pertandingan berjalan dengan seru. Team lawan berhasil dikalahkan dengan skor 1-0."
bow = dictionary.doc2bow(tokenize_and_stem(text))

print(lsi_model[bow])
print(lda_model[bow]) 

[(0, 0.12344887542630886), (1, -0.2918502993228176), (2, 0.683481199071883)]
[(0, 0.05637562), (1, 0.052879747), (2, 0.8907446)]


The LDA result can be interpreted as a distribution over topics.
Gensim offers a simple way of performing similarity queries using topic models.

In [10]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(article[document_id][:1000])

[(3, 0.9982383), (7, 0.9975495), (9, 0.99625534), (10, 0.99625534), (17, 0.99625534), (23, 0.99625534), (24, 0.99625534), (25, 0.99625534), (26, 0.99625534), (28, 0.99625534)]



Jakarta, CNN Indonesia -- Tim Detasemen Khusus 88 Antiteror Mabes Polri terus memburu jejaring kelompok teroris, selepas rentetan serangan bom yang mengguncang Kota Surabaya, Jawa Timur, beberapa waktu lalu. Salah satu orang terduga teroris dibekuk di Kabupaten Bandung Barat pada Selasa (19/6) kemarin, usai makan di sebuah rumah makan Sunda.

Sebagaimana dilansir Antara, Rabu (20/6), terduga teroris dibekuk itu adalah seorang pria berinisial F. Dia ditangkap di rumah makan khas sunda Sangkan Hurip, di Kawasan Punclut, Kabupaten Bandung Barat.

Salah seorang pegawai Rumah Makan Sangkan Hurip, Riza, mengatakan F ditangkap polisi saat hendak melaksanakan salat di musala yang disediakan oleh rumah makan itu.

"Betul, kemarin ada yang ditangkap di sini. Satu orang laki-laki," ujar Riza Putra.

Riza mengatakan, F da

Notice how the factors corresponding to each component (topic) add up to 1. That’s not a coincidence. Indeed, LDA considers documents as being generated by a mixture of the topics. The purpose of LDA is to compute how much of the document was generated by which topic. 

LDA is an iterative algorithm. Here are the two main steps:

   - In the initialization stage, each word is assigned to a random topic.
   - Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:
        - What’s the probability of the word belonging to a topic
        - What’s the probability of the document to be generated by a topic

Due to these important qualities, we can visualize LDA results easily. We’re going to use a specialized tool called PyLDAVis:

In [ ]:
import pyLDAvis.gensim
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
panel